In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [25]:
dtype = torch.FloatTensor

In [26]:
embedding_size =2 # n-gram
sequence_length = 3
num_classes = 2
filter_sizes = [2,2,2] # n-gram 다양한 filtersize로 filter를 적용하겠다. 
num_filters = 3

# 3 words sentences(=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball","i hate you","sorry for that","this is awful"]
labels = [1,1,1,0,0,0] # 1: 긍정 0: 부정

In [27]:
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}

In [28]:
word_dict

{'he': 0,
 'she': 1,
 'for': 2,
 'awful': 3,
 'loves': 4,
 'i': 5,
 'likes': 6,
 'love': 7,
 'hate': 8,
 'me': 9,
 'you': 10,
 'that': 11,
 'this': 12,
 'baseball': 13,
 'is': 14,
 'sorry': 15}

In [29]:
vocab_size = len(word_dict)

In [30]:
inputs = []
for sen in sentences:
    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))

In [31]:
inputs

[array([ 5,  7, 10]),
 array([0, 4, 9]),
 array([ 1,  6, 13]),
 array([ 5,  8, 10]),
 array([15,  2, 11]),
 array([12, 14,  3])]

In [32]:
targets = []
for out in labels:
    targets.append(out)

In [33]:
input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

In [34]:
input_batch

tensor([[ 5,  7, 10],
        [ 0,  4,  9],
        [ 1,  6, 13],
        [ 5,  8, 10],
        [15,  2, 11],
        [12, 14,  3]])

In [35]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN,self).__init__()
        
        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size,embedding_size).uniform_(-1,1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total,num_classes)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)
        
    def forward(self,X):
        embedded_chars = self.W[X] # [batch_size, sequence_length, embedding_size]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
        
        pooled_outputs =[]
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1,num_filters,(filter_size, embedding_size), bias = True)(embedded_chars) ## conv2d를 embedded_chars에 적용
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length-filter_size+1,1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0,3,2,1)
            pooled_outputs.append(pooled)
            
        h_pool = torch.cat(pooled_outputs,len(filter_sizes)) ## channel을 늘인거 [batch_size(=6), output_height(=1), output_width(=1),output_channel(=3)*3]
        h_pool_flat = torch.reshape(h_pool, [-1,self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel *3)] 여기서 output_channel *3이 결국 num_filters_total이 된다
        model = torch.mm(h_pool_flat,self.Weight) + self.Bias # [batch_size, num_clasees]    
        return model

In [36]:
model = TextCNN()

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [38]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)
    
    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    
    loss = criterion(output,target_batch)
    if (epoch + 1) % 1000 ==0:
        print('Epoch: ', '%04d' % (epoch +1), 'cost = ', '{:.6f}'.format(loss))
        
    loss.backward()
    optimizer.step()
    

    
    
    

Epoch:  1000 cost =  0.202907
Epoch:  2000 cost =  0.040229
Epoch:  3000 cost =  0.052456
Epoch:  4000 cost =  0.101490
Epoch:  5000 cost =  0.021955


In [39]:
# Test
test_text = 'sorry hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Variable(torch.LongTensor(tests))

In [40]:
# Predict
predict = model(test_batch).data.max(1,keepdim=True)[1]
if predict[0][0] ==0:
    print(test_text,"is Bad Mean...")
    
else:
    print(test_text,"is Good Mean!")

sorry hate you is Bad Mean...
